In [1848]:
import pandas as pd
import numpy as np
import debug
import cv2

In [1849]:
SIZE = 700

image = cv2.imread("../images/boards/yellow_black_up.jpeg")
image_size = (SIZE,SIZE)
image = cv2.resize(image,image_size)

In [1850]:
def _detect_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 90, 400, 3)
    return edges

edges_image = _detect_edges(image)

In [1851]:
HOUGH_THRESHOLD = 100
THETA_RESOLUTION = np.pi / 180
RHO_RESOLUTION = 1

def _generate_edges_lines(canny):
    
    lines = cv2.HoughLines(canny, 
                            rho = RHO_RESOLUTION, 
                            theta =  THETA_RESOLUTION, 
                            threshold = HOUGH_THRESHOLD)
    
    #Format dataset
    lines = lines.reshape(len(lines),2)
    
    coor_lines = []
    for line in lines:
        rho,theta = line
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 +1000*(-b))
        y1 = int(y0 +1000*(a))
        x2 = int(x0 -1000*(-b))
        y2 = int(y0 -1000*(a))   
        coor_lines.append(((x1,y1),(x2,y2)))
    
    return np.array(coor_lines)
    
lines = _generate_edges_lines(edges_image)

In [1852]:
from sklearn.cluster import KMeans

def _get_lines_means(points, direction, n_clusters = 9):
    
    means = []

    for lines in points:
    
        kmeans = KMeans(n_clusters = n_clusters, n_init=10)
        kmeans.fit(lines)
        labels = kmeans.labels_
    
        unique_labels = list(range(n_clusters))
    
        cluster_means = []
    
        for label in unique_labels: 
            cluster_data = lines[labels == label] 
            cluster_mean = np.round(sum(cluster_data)/len(cluster_data))  
            cluster_means.append(cluster_mean)
        
        cluster_means = np.array(cluster_means)
        sorted_index = np.argsort(cluster_means[:, direction])
        
        means.append(cluster_means[sorted_index])
 
    return means


In [1853]:
VERTICAL = 0
HORIZONTAL = 1

def _get_vertical_averages(lines):
    
    vertical_lines = lines[lines[:,0,1] > SIZE]
    
    initial_points = vertical_lines[:,1][vertical_lines[:,1,1] < -SIZE]
    final_points = vertical_lines[:,0][vertical_lines[:,0,1] > SIZE] 
    
    averages = _get_lines_means((initial_points, final_points), VERTICAL)
    
    return np.stack(averages, axis=1)

def _get_horizontal_averages(lines):

    horizontal_lines = lines[lines[:,1,0] > SIZE]
    
    initial_points = horizontal_lines[:,0][horizontal_lines[:,0,0] < -SIZE]
    final_points = horizontal_lines[:,1][horizontal_lines[:,1,0] > SIZE]
    
    averages = _get_lines_means((initial_points, final_points), HORIZONTAL)
    
    return np.stack(averages, axis=1)
    
vertical_lines = _get_vertical_averages(lines).astype(int)
horizontal_lines = _get_horizontal_averages(lines).astype(int)

lines = np.concatenate((vertical_lines,horizontal_lines), axis=0)

debug.lines(image,lines, size = 3);


In [1854]:
def _detect_inner_corners(lines):
    
    black_image = np.zeros((SIZE, SIZE, 3), dtype=np.uint8)
    debug.lines(black_image,lines, size = 3);
    black_image = _detect_edges(black_image)
    points = cv2.goodFeaturesToTrack(black_image, maxCorners=81, qualityLevel=.5, minDistance=30)
    return points.reshape(len(points),2)
    
points =  _detect_inner_corners(lines)
debug.points(image,points, color = (255,0,0),size = 7);


In [1855]:
"""import itertools

def line_intersections(line1, line2):
    x1, y1 = line1[0]
    x2, y2 = line1[1]
    x3, y3 = line2[0]
    x4, y4 = line2[1]

    # Calculate the denominator for the intersection point calculation
    denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if  0 in denominator:
        return None  # Lines are parallel and do not intersect

    # Calculate the intersection point
    px = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denominator
    py = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denominator

    return (px, py)

def line_intersection(lines):

    lines = list(itertools.combinations(lines, 2))
    intersections = []

    for i in range(len(lines)):
        intersection = line_intersections(lines[0], lines[1])
        if intersection is not None:
                intersections.append(intersection)
            
    return np.array(points)



#print(lines)
points = line_intersection(lines)

print(points)

lower_limit = -1000
upper_limit = 1000
#points = points.astype(int)
# Create a mask to filter points within the range
#mask = (points[:, 0] >= lower_limit) & (points[:, 0] <= upper_limit) & (points[:, 1] >= lower_limit) & (points[:, 1] <= upper_limit)

#points = points[mask]

print(len(points))
debug.points(image,points, color = (0,255,0), size = 7);

#print(len(sorted_intersections))"""

'import itertools\n\ndef line_intersections(line1, line2):\n    x1, y1 = line1[0]\n    x2, y2 = line1[1]\n    x3, y3 = line2[0]\n    x4, y4 = line2[1]\n\n    # Calculate the denominator for the intersection point calculation\n    denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)\n    if  0 in denominator:\n        return None  # Lines are parallel and do not intersect\n\n    # Calculate the intersection point\n    px = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denominator\n    py = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denominator\n\n    return (px, py)\n\ndef line_intersection(lines):\n\n    lines = list(itertools.combinations(lines, 2))\n    intersections = []\n\n    for i in range(len(lines)):\n        intersection = line_intersections(lines[0], lines[1])\n        if intersection is not None:\n                intersections.append(intersection)\n            \n    return np.array(points)\n\n\n\n#print(lines)\npoints

In [1857]:
print(points)

[[213. 111.]
 [129. 637.]
 [ 55. 637.]
 [435. 636.]
 [361. 636.]
 [285. 636.]
 [206. 636.]
 [652. 635.]
 [581. 635.]
 [ 56. 564.]
 [362. 563.]
 [286. 563.]
 [207. 563.]
 [510. 562.]
 [436. 562.]
 [653. 561.]
 [582. 561.]
 [653. 495.]
 [583. 495.]
 [510. 494.]
 [436. 494.]
 [362. 493.]
 [286. 493.]
 [208. 492.]
 [132. 492.]
 [ 57. 491.]
 [584. 419.]
 [511. 418.]
 [437. 417.]
 [363. 416.]
 [287. 414.]
 [209. 413.]
 [133. 412.]
 [ 58. 411.]
 [654. 341.]
 [585. 341.]
 [512. 340.]
 [438. 340.]
 [364. 339.]
 [288. 338.]
 [210. 338.]
 [134. 337.]
 [654. 262.]
 [513. 261.]
 [438. 260.]
 [364. 260.]
 [288. 259.]
 [211. 258.]
 [136. 258.]
 [ 60. 257.]
 [655. 190.]
 [514. 187.]
 [439. 186.]
 [365. 185.]
 [289. 184.]
 [212. 182.]
 [137. 181.]
 [ 60. 180.]
 [655. 120.]
 [514. 117.]
 [ 70. 117.]
 [440. 115.]
 [366. 114.]
 [290. 112.]
 [138. 109.]
 [656.  50.]
 [589.  49.]
 [515.  47.]
 [440.  45.]
 [366.  43.]
 [290.  42.]
 [214.  40.]
 [139.  38.]
 [ 62.  36.]
 [518. 635.]
 [130. 573.]
 [653. 429.]

In [1856]:

cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()